In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pyodbc
import time

def executar_conexao_bd():
    server = 'TBDBL-GLEISSON'
    database = 'Dados_Pavimento'
    username = 'gleisson.santos'
    password = '12345'
    conexao = pyodbc.connect(
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
    print('Executado a conexão com o BD......')
    return conexao

cursor = executar_conexao_bd()
cursor

Executado a conexão com o BD......


In [15]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 110.0.5481
[WDM] - Get LATEST chromedriver version for 110.0.5481 google-chrome
[WDM] - Driver [C:\Users\gleisson.santos\.wdm\drivers\chromedriver\win32\110.0.5481.77\chromedriver.exe] found in cache
C:\Users\GLEISS~1.SAN\AppData\Local\Temp/ipykernel_14632/3643764433.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [16]:
login = 't034183'
senha = 'CNB@2022'
driver.get("https://sciweb.embasanet.ba.gov.br/sci-web/")
time.sleep(2)

# Procurar a tag random do site / Login e senha
randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(login)
driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
driver.find_element(By.ID, "loginForm-submit").click()
time.sleep(2)

In [17]:
#tipo: insu, menu: GOSI_anchor - ACESSAR MENU DE INSUMOS
def arvore_acesso(driver, tipo, menu ):
    # Proc detalhe da Os no campo de buscar
    driver.find_element(By.ID, "arvoreSearch").send_keys(tipo)
    # aguardar ate o elemento aparecer e/ou seguir codigo
    wait = WebDriverWait(driver, 5)
    element1 = wait.until(EC.element_to_be_clickable((By.ID, menu)))
    # acessar Campo detalhe da Os
    driver.find_element(By.ID, menu).click()
    time.sleep(3)
    #acessar camada frame do site
    driver.switch_to.frame("frame-content")

#ACESSAR MENU PAR EXTRAR INSUMADOS
arvore_acesso(driver, 'detalhe', 'DOSE_anchor')

### Acesso so item "Detalhes da Os"

In [18]:
#Consultar na label Material
def preencher_ss(driver, lista, lista2):
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt71-item"]').clear()
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt79-item"]').clear()
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt71-item"]').send_keys(lista)
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt79-item"]').send_keys(lista2)
    time.sleep(0.5)
    # clinical em buscar
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-panel"]/input').click()
    #coletar valor solicitado
    
    try:
        material = driver.find_element(By.CSS_SELECTOR, "label[title='Material Utilizado'] ").text
    except NoSuchElementException:
        material = 'None'
    
    try:
        hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do HD'] ").text
    except NoSuchElementException:
        try:
            hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do HD Instalado'] ").text
        except NoSuchElementException:
            try:
                hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do Hidrômetro Instalado'] ").text
            except NoSuchElementException:
                try:
                    hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do Hidrômetro'] ").text
                except NoSuchElementException:
                    try:
                        hd = driver.find_element(By.CSS_SELECTOR, "label[title='Hidrômetro Instalado'] ").text
                    except NoSuchElementException:
                        hd = None

    return material, hd


In [19]:
# df = pd.read_sql('SELECT  *  FROM  [Dados_Pavimento].[dbo].[SCI Web]', executar_conexao())
df = pd.read_sql('SELECT * FROM [Janeiro26/12-25/01]', executar_conexao_bd())
df = pd.DataFrame(df, columns = ['SS', 'OS', 'Material', 'Mureta','Hd'])
#df = df.astype(str)

display(df)

Executado a conexão com o BD......


,SS,OS,Material,Mureta,Hd
0,979395105,1,None,None,None
1,979395691,1,None,None,None
2,979395936,1,None,None,None
3,979399380,1,None,None,None
4,979399810,1,None,None,None
...,...,...,...,...,...
1420,979391934,1,None,None,None
1421,979391640,2,None,None,None
1422,979392865,1,None,None,None
1423,979393301,1,None,None,None


In [20]:
#Extrair valores e criar coluna Material insumado
dados = []
hds = []
for i, row in df.iterrows():
    a = preencher_ss(driver, row['SS'], row['OS'])
    time.sleep(0.3)
    dados.append(a[0])
    hds.append(a[1])
df['Material'] = dados
df['Hd'] = hds


#Acessar tabela do BD e extrair apenas item do texto
m_encontrado = ['Mureta']

material = []

for i, row in df.iterrows():
    if any(x in row['Material'] for x in m_encontrado):
        material.append('Sim')
    else:
        material.append('Não')     
df['Mureta'] = material


df = df.astype(str)
df = pd.DataFrame(df, columns = ['SS', 'OS', 'Material', 'Mureta','Hd'])
display(df)



NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.178)
Stacktrace:
Backtrace:
	(No symbol) [0x00A637D3]
	(No symbol) [0x009F8B81]
	(No symbol) [0x008FB36D]
	(No symbol) [0x008DE5DA]
	(No symbol) [0x0094F0CB]
	(No symbol) [0x00960A56]
	(No symbol) [0x0094B216]
	(No symbol) [0x00920D97]
	(No symbol) [0x0092253D]
	GetHandleVerifier [0x00CDABF2+2510930]
	GetHandleVerifier [0x00D08EC1+2700065]
	GetHandleVerifier [0x00D0C86C+2714828]
	GetHandleVerifier [0x00B13480+645344]
	(No symbol) [0x00A00FD2]
	(No symbol) [0x00A06C68]
	(No symbol) [0x00A06D4B]
	(No symbol) [0x00A10D6B]
	BaseThreadInitThunk [0x75AD00F9+25]
	RtlGetAppContainerNamedObjectPath [0x77597BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77597B8E+238]


In [12]:
display(df["Material"])

0                                                       
1                                                       
2                                                       
3                  1 HIDRO VZ 3M 1 REGISTRO PEAD DN 1/2.
4                  1 HIDRO VZ 3M 1 REGISTRO PEAD DN 1/2.
                             ...                        
750                                                     
751                                                     
752                                                     
753    2 ADAPTADOR MACHO PEAD 20MM 2 LUVA 20X1/2 20 C...
754    1 LUVA PVC DN 85 1 COLAR 85X1/2 2 ADAPTADOR 20...
Name: Material, Length: 755, dtype: object

In [13]:
for index, row in df.iterrows():    
    ID = row.Material
    SS = row.SS
    MT = row.Mureta
    HD = row.Hd
    comando_sql = (f"UPDATE SCIWeb SET Material='{ID}', Mureta='{MT}', Hd='{HD}' WHERE SS ={SS}")
    cursor.execute(comando_sql)
    
cursor.commit()

In [123]:
ss = 972311398
value_column = 'Insumado'
ID = row.Insumado
SS = row.SS

for index, row in df.iterrows():
    comando_sql = (f"UPDATE SCIWeb SET Insumado = 'Testando' WHERE OS = 1 ")

cursor.execute(comando_sql)
cursor.commit()

In [2]:
#Limpar colunas do Banco de dados;

df = df.astype(str)
df = pd.DataFrame(df, columns=['SS', 'OS', 'Matutilizado', 'Hdutilizado', 'Usoumureta'])

cursor = executar_conexao_bd().cursor()
for index, row in df.iterrows():
    Mat = row.Matutilizado
    HD  = row.Hdutilizado
    INS = row.Usoumureta
    SS  = row.SS
    comando_sql = (f"UPDATE Material SET Matutilizado='NULL', Hdutilizado='NULL', Usoumureta='NULL' WHERE SS ={SS}")
    cursor.execute(comando_sql)
    
cursor.commit()
cursor.close()